In [3]:
import numpy as np

Agregar tambien un modelo que haga un promedio de las Yes


%load_ext autoreload
%autoreload 2

In [77]:
def train_test_split(x,train_pct):
  registros = len(x) # Cantidad de registros
  x = np.random.permutation(x) # Permuta el dataset
  trains = int(np.floor(train_pct*registros)) # Calcula el corte para 70%
  test_pct = 1-train_pct
  test = int(np.floor(test_pct*registros)) #idem
  x_train = x[0:trains] #toma el primero 70% de los registros permutados
  x_test = x[trains:] #idem 20&
  return x_train, x_test #devuelve los 3 splits

In [13]:
path = '/content/drive/My Drive/Especializacion UBA/Introduccion a IA/data/income_data.csv'

In [41]:
structure = [('register',np.int),
          ('income', np.float32),
          ('happiness', np.float32)]
structure = np.dtype(structure)
with open(path, encoding="utf8") as rawdataset:
      line = rawdataset.readline()
      b = rawdataset.readline()
      #line.strip() 
      rows_gen = ((np.int(line.split(',')[0].replace('"','')), line.split(',')[1],line.split(',')[2])
                        for line in rawdataset)
      registers = np.fromiter(rows_gen, structure)

In [275]:
class dataset(object):

  def __init__(self,path):
    #self.path = path
    self.dataset = self.build_dataset(path=path)
      
  def split(self,percentage):
    #returns train test segun el parametro percentaje
    self.train,self.test = train_test_split(self.dataset,percentage)
    
  def build_dataset(self):
    #usar np estructurado para levantar la data y cargarla
    #Agregar opcion header y estructura dinamica
    structure = [('register', np.int),
              ('income', np.float32),
              ('happiness', np.float32)]
    structure = np.dtype(structure)
    # load numpy array from disk using a generator
    with open(path, encoding="utf8") as rawdataset:
      line = rawdataset.readline()
      line.strip()
      rows_gen = ((np.int(line.split(',')[0].replace('"','')), line.split(',')[1],line.split(',')[2])
                        for line in rawdataset)
      loadedData = np.fromiter(rows_gen, structure)

    return loadedData

In [122]:
x = datos['income']
y = datos['happiness']

In [141]:
class BaseModel(object):
  def __init__(self):
    self.model = None

  def fit(self,x,y):
    #train del model
    return NotImplemented

  def predict(self,x):
    #returns yhat
    return NotImplemented

In [214]:
class LinearRegression(BaseModel):
  def fit(self,x,y):
    #self.model= W 
    # c'alcular W y guardarlo en el model
    if len(y.shape == 1):
      y = y[None].T #Le agrego una dimensión y lo hago vector columna

    if len(x.shape) == 1:
      x = x[None].T # Le agrego la dimensión y lo hago vector columna
    else:
      x = x.T
    
    xt = x.T # Transpongo, queda vector fila
    xty = np.matmul(xt,y)
    inv_xtx = np.linalg.inv(np.matmul(xt,x))
    self.w = np.matmul(inv_xtx,xty)
    
  def predict(self,X):
    #usar el modelo para predecir y hat a partir de X y W
    if len(X.shape) == 1:
      X = x[None].T
    y_predict = np.matmul(self.w.T,X)  
    return y_predict

In [313]:
class AverageBaseModel(BaseModel):
  def __init__(self,y):
    self.y = y
  def predict(self,x):
    y_mean = np.mean(self.y)
    n = len(x)
    prediction = y_mean * np.ones(shape=(n,1))
    return prediction

In [263]:
class LinearRegressionAffine(BaseModel):
  def fit(self,x,y):
    #self.model= W 
    # c'alcular W y guardarlo en el model
    if len(y.shape) == 1:
      y = y[None].T #Le agrego una dimensión y lo hago vector columna

    if len(x.shape) == 1:
      x = x[None].T # Le agrego la dimensión y lo hago vector columna
    else:
      x = x.T
    n = len(x)
    X0 = np.ones((n,1))
    x = np.hstack((x,X0))
    xt = x.T # Transpongo, queda vector fila
    xty = np.matmul(xt,y)
    inv_xtx = np.linalg.inv(np.matmul(xt,x))
    self.w = np.matmul(inv_xtx,xty)
        
  def predict(self,X):
    #usar el modelo para predecir y hat a partir de X y W
    if len(X.shape) == 1:
      X = X[None].T
    n = len(X)
    X0 = np.ones((n,1))
    self.X = np.hstack((X,X0))
    y_predict = np.matmul(self.X,self.w)
    return y_predict

In [301]:
class Metric(object):
  def __call__(self,target,prediction):
    return NotImplemented

class MSE(Metric):
  def __call__(self,target,prediction):
    #Flattens para asegurar que tengan shape (len,)
    prediction = prediction.flatten()
    target = target.flatten()
    # Calcula el mse y lo devuelve      
    mse = np.sum((target - prediction)**2)
    return mse

#Prueba del funcionamiento de todo

In [276]:
path

'/content/drive/My Drive/Especializacion UBA/Introduccion a IA/data/income_data.csv'

In [277]:
datos = dataset(path=path,percentage=0.80)

In [278]:
datos.split()

In [279]:
lr = LinearRegressionAffine()

In [280]:
x = datos.train['income']
y = datos.train['happiness']

In [281]:
lr.fit(x=x,y=y)

In [282]:
pred = lr.predict(X=datos.test['income'])

In [302]:
mse = MSE()

In [306]:
mse(target=datos.test['happiness'],prediction=pred)

45.135487852605145

In [314]:
modelo_base = AverageBaseModel(datos.test['happiness'])

In [317]:
basePrediction = modelo_base.predict(datos.test['happiness'])

In [318]:
mse(target=datos.test['happiness'],prediction=basePrediction)

187.5630869794794

Se verifica que la Affine Linear Regresion tiene un MSE menor que la Base Prediction. 